# Sudoku Example

Lucerne University of Applied Sciences and Arts - School of Information Technology

Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.

@author: Tobias Mérinat

Imports

In [15]:
from ortools.constraint_solver import pywrapcp
from itertools import product

Pretty-print square sudoku board

In [16]:
def pretty_print(board):
    for i in range(len(board)):
        for j in range(len(board)):
            print("[{}] ".format(board[i][j].Value()), end='')
        print("\n")
    print("\n\n")

Sudoku puzzles from newspapers. Get more from [here](https://sudoku.tagesspiegel.de/sudoku-sehr-schwer/)

In [17]:
sudoku1 = [
    ["1", "", "", "", "3", "", "", "8", ""],
    ["", "6", "", "4", "", "", "", "", ""],
    ["", "", "4", "", "", "9", "3", "", ""],
    ["", "4", "5", "", "", "6", "", "", "7"],
    ["9", "", "", "", "", "5", "", "", ""],
    ["", "", "8", "", "", "3", "", "2", ""],
    ["", "", "", "", "", "", "9", "5", "6"],
    ["", "2", "", "", "", "", "", "", ""],
    ["", "", "7", "", "", "8", "", "1", ""]
]
sudoku2 = [
    ["4", "", "8", "", "", "", "", "", ""],
    ["", "", "", "1", "7", "", "", "", ""],
    ["", "", "", "", "8", "", "", "3", "2"],
    ["", "", "6", "", "", "8", "2", "5", ""],
    ["", "9", "", "", "", "", "", "8", ""],
    ["", "3", "7", "6", "", "", "9", "", ""],
    ["2", "7", "", "", "5", "", "", "", ""],
    ["", "", "", "", "1", "4", "", "", ""],
    ["", "", "", "", "", "", "6", "", "4"]
]
sudoku3 = [
    ["8", "", "", "3", "", "2", "", "", "7"],
    ["", "4", "", "", "6", "", "", "9", ""],
    ["", "", "5", "", "", "", "6", "", ""],
    ["1", "", "", "6", "", "8", "", "", "5"],
    ["", "3", "", "", "2", "", "", "1", ""],
    ["4", "", "", "7", "", "3", "", "", "6"],
    ["", "", "6", "", "", "", "8", "", ""],
    ["", "2", "", "", "3", "", "", "6", ""],
    ["5", "", "", "2", "", "6", "", "", "1"]
]
sudoku4 = [
    ["", "", "6", "1", "", "", "", "", "8"],
    ["", "7", "", "", "9", "", "", "2", ""],
    ["3", "", "", "", "", "6", "9", "", ""],
    ["6", "", "", "", "", "2", "3", "", ""],
    ["", "8", "", "", "4", "", "", "1", ""],
    ["", "", "4", "3", "", "", "", "", "9"],
    ["", "", "9", "2", "", "", "", "", "4"],
    ["", "5", "", "", "7", "", "", "8", ""],
    ["8", "", "", "", "", "5", "1", "", ""]
]

Pick one

In [18]:
game = sudoku1

Create constraint solver

In [19]:
solver = pywrapcp.Solver("Sudoku")

Board configuration (should match game)

In [20]:
cell_size = 3
board_size = cell_size * cell_size
board_indices = list(product(range(board_size), repeat=2))  # tuples (i, j) for all board indices
cell_indices = list(product(range(cell_size), repeat=2))  # tuples (i, j) for all cell indices

Board as nxn matrix of decision variables with value in {1..9]

In [21]:
board = [[solver.IntVar(1, board_size) for _j in range(board_size)] for _i in range(board_size)]

Pre-Assignments

In [22]:
for i, j in board_indices:
    if len(game[i][j]) > 0:  # elements are strings, so check for length
        solver.Add(board[i][j] == int(game[i][j]))

Each row and each column contains only different values

In [23]:
for i in range(board_size):
    solver.Add(solver.AllDifferent([board[i][j] for j in range(board_size)]))  # Rows
    solver.Add(solver.AllDifferent([board[j][i] for j in range(board_size)]))  # Columns

Each 3x3 sub-matrix contains only different values

In [24]:
for i, j in cell_indices:
    solver.Add(solver.AllDifferent(
        [board[i * cell_size + di][j * cell_size + dj] for di in range(cell_size) for dj in range(cell_size)]))

Configure solver

In [25]:
db = solver.Phase([board[i][j] for i, j in board_indices], solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [26]:
solver.NewSearch(db)
while solver.NextSolution():
    pretty_print(board)

[1] [5] [9] [6] [3] [2] [7] [8] [4] 

[3] [6] [2] [4] [8] [7] [1] [9] [5] 

[7] [8] [4] [1] [5] [9] [3] [6] [2] 

[2] [4] [5] [9] [1] [6] [8] [3] [7] 

[9] [7] [3] [8] [2] [5] [6] [4] [1] 

[6] [1] [8] [7] [4] [3] [5] [2] [9] 

[8] [3] [1] [2] [7] [4] [9] [5] [6] 

[5] [2] [6] [3] [9] [1] [4] [7] [8] 

[4] [9] [7] [5] [6] [8] [2] [1] [3] 






Cleanup

In [27]:
solver.EndSearch()

Print solver information

In [28]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   50ms
Failures:  13
Branches:  26 
